In [0]:

from pyspark.sql import functions as F, DataFrame
from pyspark.sql.types import *
from pyspark.sql.window import Window
from typing import Dict, List, Tuple, Optional
from datetime import datetime, date
import json

# Para visualizaciones
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
except:
    print("⚠️ Matplotlib/Seaborn no disponibles. Instalando...")
    %pip install matplotlib seaborn

# COMMAND ----------

# MAGIC %md
# MAGIC ## 🎯 Clase de Configuración y Constantes

# COMMAND ----------

class Config:
    """Configuración centralizada"""
    
    # Tablas
    class Tables:
        MATERIALES = 'damm_silver_des.dm.dm_material_1'
        VENTAS = 'damm_silver_des.gest_com_horeca.ca_crm_ventas_posicion'
        RVL = 'damm_silver_des.gest_com_horeca.ca_crm_rentabilidad'
        OUTPUT = 'damm_silver_des.pricing_unho.rentabilidad_consolidada'  
    
    # Parámetros de validación
    FECHA_INICIO = "2024-01-01"
    FECHA_FIN = "2025-12-31"
    SECTOR_ESPERADO = '1'
    MATERIALES_EXCLUIDOS = ["TB8", "TB10", "TB12"]
    
    # Umbrales de calidad
    class Thresholds:
        MAX_NULL_PERCENTAGE = 0.05  # 5% máximo de nulls permitido
        MAX_DUPLICATE_PERCENTAGE = 0.01  # 1% máximo de duplicados
        MAX_RECORD_LOSS_PERCENTAGE = 0.001  # 0.1% pérdida máxima en joins
        MIN_RECORDS_EXPECTED = 1000  # Mínimo de registros esperados
        OUTLIER_ZSCORE_THRESHOLD = 3  # Z-score para detección de outliers

class ClasesCondiciones:
    """Clasificación de condiciones comerciales"""
    Tarifa = ["050"]
    Obsequios = ["100"]
    Promociones = ["300"]
    Descuentos = ["210"]
    Contratos = ["400"]
    AmortizacionesCDT = ["420"]
    Rappels = ["740"]
    CostesDistribuidor = ["900", "901", "920", "902", "930"]
    Costes = ["858", "859", "861", "890", "954", "956"]
    
    @classmethod
    def get_all_codes(cls) -> List[str]:
        """Retorna todos los códigos definidos"""
        all_codes = []
        for attr_name in dir(cls):
            if not attr_name.startswith('_') and attr_name != 'get_all_codes':
                attr_value = getattr(cls, attr_name)
                if isinstance(attr_value, list):
                    all_codes.extend(attr_value)
        return all_codes

# COMMAND ----------

# MAGIC %md
# MAGIC ## 🛠️ Clase de Utilidades para Data Quality

# COMMAND ----------

class DataQualityChecker:
    """Clase para realizar chequeos de calidad de datos"""
    
    def __init__(self, spark):
        self.spark = spark
        self.results = []
        self.test_count = 0
        self.passed_count = 0
        self.failed_count = 0
    
    def log_test(self, test_name: str, status: str, message: str, 
                 severity: str = "ERROR", details: Dict = None):
        """Registra resultado de un test"""
        self.test_count += 1
        if status == "PASS":
            self.passed_count += 1
            icon = "✅"
        else:
            self.failed_count += 1
            icon = "❌"
        
        result = {
            "timestamp": datetime.now().isoformat(),
            "test_number": self.test_count,
            "test_name": test_name,
            "status": status,
            "severity": severity,
            "message": message,
            "details": details or {}
        }
        
        self.results.append(result)
        print(f"{icon} Test #{self.test_count}: {test_name} - {status}")
        print(f"   {message}")
        if details:
            print(f"   Details: {json.dumps(details, indent=2)}")
        print()
    
    def get_summary(self) -> Dict:
        """Retorna resumen de tests"""
        return {
            "total_tests": self.test_count,
            "passed": self.passed_count,
            "failed": self.failed_count,
            "success_rate": f"{(self.passed_count/self.test_count*100):.2f}%" if self.test_count > 0 else "0%"
        }
    
    def display_summary(self):
        """Muestra resumen visual"""
        summary = self.get_summary()
        print("=" * 80)
        print("📊 DATA QUALITY TEST SUMMARY")
        print("=" * 80)
        print(f"Total Tests Run: {summary['total_tests']}")
        print(f"✅ Passed: {summary['passed']}")
        print(f"❌ Failed: {summary['failed']}")
        print(f"Success Rate: {summary['success_rate']}")
        print("=" * 80)
        
        # Crear DataFrame de resultados
        if self.results:
            df_results = spark.createDataFrame(self.results)
            display(df_results)
    
    def check_null_percentage(self, df: DataFrame, column: str, 
                             threshold: float, test_name: str = None) -> bool:
        """Valida que el porcentaje de nulls no supere el umbral"""
        if test_name is None:
            test_name = f"Null Check - {column}"
        
        total_count = df.count()
        null_count = df.filter(F.col(column).isNull()).count()
        null_percentage = (null_count / total_count * 100) if total_count > 0 else 0
        
        passed = null_percentage <= (threshold * 100)
        status = "PASS" if passed else "FAIL"
        
        message = f"Column '{column}': {null_percentage:.2f}% nulls (threshold: {threshold*100}%)"
        details = {
            "column": column,
            "total_records": total_count,
            "null_count": null_count,
            "null_percentage": f"{null_percentage:.2f}%",
            "threshold": f"{threshold*100}%"
        }
        
        self.log_test(test_name, status, message, details=details)
        return passed
    
    def check_duplicates(self, df: DataFrame, key_columns: List[str], 
                        threshold: float, test_name: str = None) -> bool:
        """Valida que no haya duplicados en las columnas clave"""
        if test_name is None:
            test_name = f"Duplicate Check - {', '.join(key_columns)}"
        
        total_count = df.count()
        distinct_count = df.select(key_columns).distinct().count()
        duplicate_count = total_count - distinct_count
        duplicate_percentage = (duplicate_count / total_count * 100) if total_count > 0 else 0
        
        passed = duplicate_percentage <= (threshold * 100)
        status = "PASS" if passed else "FAIL"
        
        message = f"Keys {key_columns}: {duplicate_percentage:.2f}% duplicates (threshold: {threshold*100}%)"
        details = {
            "key_columns": key_columns,
            "total_records": total_count,
            "distinct_records": distinct_count,
            "duplicates": duplicate_count,
            "duplicate_percentage": f"{duplicate_percentage:.2f}%",
            "threshold": f"{threshold*100}%"
        }
        
        self.log_test(test_name, status, message, details=details)
        return passed
    
    def check_record_count(self, df: DataFrame, min_expected: int, 
                          test_name: str = None) -> bool:
        """Valida que el dataframe tenga al menos el número mínimo de registros"""
        if test_name is None:
            test_name = "Minimum Record Count Check"
        
        actual_count = df.count()
        passed = actual_count >= min_expected
        status = "PASS" if passed else "FAIL"
        
        message = f"Record count: {actual_count:,} (minimum expected: {min_expected:,})"
        details = {
            "actual_count": actual_count,
            "min_expected": min_expected,
            "difference": actual_count - min_expected
        }
        
        self.log_test(test_name, status, message, details=details)
        return passed
    
    def check_value_in_list(self, df: DataFrame, column: str, 
                           valid_values: List, test_name: str = None) -> bool:
        """Valida que todos los valores de una columna estén en una lista válida"""
        if test_name is None:
            test_name = f"Valid Values Check - {column}"
        
        invalid_values = (
            df.filter(~F.col(column).isin(valid_values))
            .select(column)
            .distinct()
            .collect()
        )
        
        passed = len(invalid_values) == 0
        status = "PASS" if passed else "FAIL"
        
        invalid_list = [row[column] for row in invalid_values]
        message = f"Column '{column}': Found {len(invalid_values)} invalid values"
        details = {
            "column": column,
            "valid_values": valid_values,
            "invalid_values": invalid_list[:10],  # Mostrar máximo 10
            "total_invalid": len(invalid_values)
        }
        
        self.log_test(test_name, status, message, details=details)
        return passed
    
    def compare_row_counts(self, df1: DataFrame, df2: DataFrame, 
                          df1_name: str, df2_name: str, 
                          threshold: float, test_name: str = None) -> bool:
        """Compara conteos de registros entre dos dataframes"""
        if test_name is None:
            test_name = f"Row Count Comparison: {df1_name} vs {df2_name}"
        
        count1 = df1.count()
        count2 = df2.count()
        
        if count1 > 0:
            diff_percentage = abs(count1 - count2) / count1 * 100
        else:
            diff_percentage = 100 if count2 > 0 else 0
        
        passed = diff_percentage <= (threshold * 100)
        status = "PASS" if passed else "FAIL"
        
        message = f"{df1_name}: {count1:,} | {df2_name}: {count2:,} | Diff: {diff_percentage:.2f}%"
        details = {
            df1_name: count1,
            df2_name: count2,
            "difference": count1 - count2,
            "diff_percentage": f"{diff_percentage:.2f}%",
            "threshold": f"{threshold*100}%"
        }
        
        self.log_test(test_name, status, message, details=details)
        return passed

# COMMAND ----------

# MAGIC %md
# MAGIC ## 1️⃣ VALIDACIÓN DE FUENTES (Source Validation)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 1.1 Validar Tabla de Materiales

# COMMAND ----------

print("🔍 Validando tabla de MATERIALES...")
print("=" * 80)

dq = DataQualityChecker(spark)

# Cargar tabla de materiales SIN filtros
df_materiales_raw = spark.table(Config.Tables.MATERIALES)

# Test 5: Verificar que los materiales excluidos existen en la tabla
materiales_excluidos_en_tabla = (
    df_materiales_raw
    .filter(F.col("Material").isin(Config.MATERIALES_EXCLUIDOS))
    .count()
)

# Aplicar filtros como en el código original
df_materiales_filtered = (
    df_materiales_raw
    .filter(F.col("Sector") == Config.SECTOR_ESPERADO)
    .filter(~F.col("Material").isin(Config.MATERIALES_EXCLUIDOS))
    .select("Material")
    .distinct()
)

# Test 6: Verificar que después del filtro quedan materiales
materiales_count = df_materiales_filtered.count()
dq.check_record_count(
    df_materiales_filtered,
    min_expected=Config.Thresholds.MIN_RECORDS_EXPECTED,
    test_name="Materiales - Después de Filtros"
)

print(f"\n📊 Materiales después de filtros: {materiales_count:,}")

# COMMAND ----------

# MAGIC %md
# MAGIC ### 1.2 Validar Tabla de Rentabilidad (RVL)

# COMMAND ----------

print("🔍 Validando tabla de RENTABILIDAD (RVL)...")
print("=" * 80)

# Cargar RVL con filtro de fechas
df_rvl_raw = (
    spark.table(Config.Tables.RVL)
    .filter(F.col("Fecha").between(Config.FECHA_INICIO, Config.FECHA_FIN))
)

# Test 1: Verificar que hay datos en el rango de fechas
dq.check_record_count(
    df_rvl_raw,
    min_expected=Config.Thresholds.MIN_RECORDS_EXPECTED,
    test_name="RVL - Datos en Rango de Fechas"
)

# Test 2: Verificar columnas clave sin nulls
for col in ["Detallista", "Material", "Fecha", "claseOperacion"]:
    dq.check_null_percentage(
        df_rvl_raw,
        column=col,
        threshold=Config.Thresholds.MAX_NULL_PERCENTAGE,
        test_name=f"RVL - {col} Nulls"
    )

# Test 3: Verificar rango de fechas
from datetime import datetime

fecha_min = df_rvl_raw.agg(F.min("Fecha")).collect()[0][0]
fecha_max = df_rvl_raw.agg(F.max("Fecha")).collect()[0][0]

# Convert config dates to datetime.date
fecha_inicio_dt = datetime.strptime(Config.FECHA_INICIO, "%Y-%m-%d").date()
fecha_fin_dt = datetime.strptime(Config.FECHA_FIN, "%Y-%m-%d").date()

if fecha_min >= fecha_inicio_dt and fecha_max <= fecha_fin_dt:
    dq.log_test(
        "RVL - Rango de Fechas Correcto",
        "PASS",
        f"Fechas en rango: {fecha_min} a {fecha_max}",
        details={"fecha_min": str(fecha_min), "fecha_max": str(fecha_max)}
    )
else:
    dq.log_test(
        "RVL - Rango de Fechas Correcto",
        "FAIL",
        f"Fechas fuera de rango esperado: {fecha_min} a {fecha_max}",
        details={
            "fecha_min_actual": str(fecha_min),
            "fecha_max_actual": str(fecha_max),
            "fecha_inicio_esperada": Config.FECHA_INICIO,
            "fecha_fin_esperada": Config.FECHA_FIN
        }
    )
# Test 4: Verificar que CtaResNiv2 contiene códigos conocidos
codigos_conocidos = ClasesCondiciones.get_all_codes()
codigos_no_mapeados = (
    df_rvl_raw
    .select("CtaResNiv2")
    .distinct()
    .filter(~F.col("CtaResNiv2").isin(codigos_conocidos))
    .collect()
)

if len(codigos_no_mapeados) == 0:
    dq.log_test(
        "RVL - Todos los CtaResNiv2 Mapeados",
        "PASS",
        "Todos los códigos están definidos en ClasesCondiciones",
        details={"codigos_conocidos": len(codigos_conocidos)}
    )
else:
    codigos_list = [row["CtaResNiv2"] for row in codigos_no_mapeados]
    dq.log_test(
        "RVL - Todos los CtaResNiv2 Mapeados",
        "FAIL",
        f"Se encontraron {len(codigos_no_mapeados)} códigos sin mapear",
        severity="WARNING",
        details={
            "codigos_no_mapeados": codigos_list[:20],  # Mostrar máximo 20
            "total_no_mapeados": len(codigos_no_mapeados)
        }
    )

print(f"\n📊 Registros RVL en rango: {df_rvl_raw.count():,}")

# COMMAND ----------

# MAGIC %md
# MAGIC ### 1.3 Validar Tabla de Ventas

# COMMAND ----------

print("🔍 Validando tabla de VENTAS...")
print("=" * 80)

df_ventas_raw = spark.table(Config.Tables.VENTAS)

# Test 1: Tabla existe y tiene datos
dq.check_record_count(
    df_ventas_raw,
    min_expected=Config.Thresholds.MIN_RECORDS_EXPECTED,
    test_name="Ventas - Tabla Existe"
)

# Test 2: Columnas clave sin nulls
for col in ["Detallista", "Material", "Fecha", "ClaseOperacionID"]:
    dq.check_null_percentage(
        df_ventas_raw,
        column=col,
        threshold=Config.Thresholds.MAX_NULL_PERCENTAGE,
        test_name=f"Ventas - {col} Nulls"
    )

# Test 3: VentaNeta y CantidadLitros no deben ser todos nulls
venta_neta_nulls = df_ventas_raw.filter(F.col("VentaNeta").isNull()).count()
cantidad_litros_nulls = df_ventas_raw.filter(F.col("CantidadLitros").isNull()).count()
total_ventas = df_ventas_raw.count()

if venta_neta_nulls < total_ventas:
    dq.log_test(
        "Ventas - VentaNeta tiene valores",
        "PASS",
        f"VentaNeta: {venta_neta_nulls:,} nulls de {total_ventas:,}",
        details={"nulls": venta_neta_nulls, "total": total_ventas}
    )
else:
    dq.log_test(
        "Ventas - VentaNeta tiene valores",
        "FAIL",
        "VentaNeta está completamente nula",
        details={"nulls": venta_neta_nulls, "total": total_ventas}
    )

print(f"\n📊 Registros Ventas: {total_ventas:,}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 2️⃣ VALIDACIÓN DE TRANSFORMACIONES

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.1 Validar Join: RVL + Materiales

# COMMAND ----------

print("🔍 Validando JOIN: RVL + Materiales...")
print("=" * 80)

# Contar registros ANTES del join
count_rvl_antes = df_rvl_raw.count()

# Realizar el join (inner join)
df_rvl_con_materiales = df_rvl_raw.join(
    df_materiales_filtered,
    on="Material",
    how="inner"
)

count_rvl_despues = df_rvl_con_materiales.count()

# Test 1: Verificar pérdida de registros en el join
perdida_registros = count_rvl_antes - count_rvl_despues
perdida_porcentaje = (perdida_registros / count_rvl_antes * 100) if count_rvl_antes > 0 else 0

if perdida_porcentaje <= Config.Thresholds.MAX_RECORD_LOSS_PERCENTAGE * 100:
    dq.log_test(
        "Join RVL+Materiales - Pérdida de Registros",
        "PASS",
        f"Pérdida: {perdida_registros:,} registros ({perdida_porcentaje:.4f}%)",
        details={
            "antes_join": count_rvl_antes,
            "despues_join": count_rvl_despues,
            "perdida_registros": perdida_registros,
            "perdida_porcentaje": f"{perdida_porcentaje:.4f}%"
        }
    )
else:
    dq.log_test(
        "Join RVL+Materiales - Pérdida de Registros",
        "FAIL",
        f"Pérdida significativa: {perdida_registros:,} registros ({perdida_porcentaje:.2f}%)",
        details={
            "antes_join": count_rvl_antes,
            "despues_join": count_rvl_despues,
            "perdida_registros": perdida_registros,
            "perdida_porcentaje": f"{perdida_porcentaje:.2f}%",
            "threshold": f"{Config.Thresholds.MAX_RECORD_LOSS_PERCENTAGE*100}%"
        }
    )

# Test 2: Verificar que no hay materiales fuera del filtro
materiales_invalidos = (
    df_rvl_con_materiales
    .join(df_materiales_filtered, on="Material", how="left_anti")
    .count()
)

if materiales_invalidos == 0:
    dq.log_test(
        "Join RVL+Materiales - Sin Materiales Inválidos",
        "PASS",
        "Todos los materiales son válidos después del join"
    )
else:
    dq.log_test(
        "Join RVL+Materiales - Sin Materiales Inválidos",
        "FAIL",
        f"Se encontraron {materiales_invalidos} registros con materiales inválidos",
        details={"materiales_invalidos": materiales_invalidos}
    )

print(f"\n📊 Registros RVL antes del join: {count_rvl_antes:,}")
print(f"📊 Registros RVL después del join: {count_rvl_despues:,}")
print(f"📊 Pérdida: {perdida_registros:,} ({perdida_porcentaje:.4f}%)")

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.2 Validar Creación de AgregadorCondicion

# COMMAND ----------

print("🔍 Validando creación de AgregadorCondicion...")
print("=" * 80)

def create_aggregator_column(col_name):
    """Replica la función del código original"""
    condition = None
    for categoria, codigos in ClasesCondiciones.__dict__.items():
        if not categoria.startswith("__") and not categoria.startswith("get_"):
            if condition is None:
                condition = F.when(F.col(col_name).isin(codigos), F.lit(categoria))
            else:
                condition = condition.when(F.col(col_name).isin(codigos), F.lit(categoria))
    return condition.otherwise(F.lit(None))

df_rvl_con_agregador = df_rvl_con_materiales.withColumn(
    "AgregadorCondicion",
    create_aggregator_column("CtaResNiv2")
)

# Test 1: Verificar que AgregadorCondicion no es null para códigos conocidos
codigos_conocidos = ClasesCondiciones.get_all_codes()
registros_con_codigo_conocido = df_rvl_con_agregador.filter(
    F.col("CtaResNiv2").isin(codigos_conocidos)
).count()

registros_con_agregador_null = df_rvl_con_agregador.filter(
    F.col("CtaResNiv2").isin(codigos_conocidos) & F.col("AgregadorCondicion").isNull()
).count()

if registros_con_agregador_null == 0:
    dq.log_test(
        "AgregadorCondicion - Sin Nulls para Códigos Conocidos",
        "PASS",
        f"Todos los {registros_con_codigo_conocido:,} registros con código conocido tienen AgregadorCondicion"
    )
else:
    dq.log_test(
        "AgregadorCondicion - Sin Nulls para Códigos Conocidos",
        "FAIL",
        f"{registros_con_agregador_null:,} registros con código conocido tienen AgregadorCondicion=null",
        details={
            "registros_con_codigo_conocido": registros_con_codigo_conocido,
            "registros_null": registros_con_agregador_null
        }
    )

# Test 2: Verificar distribución de AgregadorCondicion
distribucion = (
    df_rvl_con_agregador
    .groupBy("AgregadorCondicion")
    .agg(F.count("*").alias("count"))
    .orderBy(F.desc("count"))
)

print("\n📊 Distribución de AgregadorCondicion:")
display(distribucion)

# Test 3: Verificar que cada código se mapea a una única categoría
codigos_con_multiple_categoria = []
for categoria, codigos in ClasesCondiciones.__dict__.items():
    if not categoria.startswith("__") and not categoria.startswith("get_"):
        for codigo in codigos:
            # Verificar si el código está en otras categorías
            apariciones = 0
            for cat2, codes2 in ClasesCondiciones.__dict__.items():
                if not cat2.startswith("__") and not cat2.startswith("get_"):
                    if codigo in codes2:
                        apariciones += 1
            if apariciones > 1:
                codigos_con_multiple_categoria.append(codigo)

if len(codigos_con_multiple_categoria) == 0:
    dq.log_test(
        "AgregadorCondicion - Códigos Únicos por Categoría",
        "PASS",
        "Cada código pertenece a una única categoría"
    )
else:
    dq.log_test(
        "AgregadorCondicion - Códigos Únicos por Categoría",
        "FAIL",
        f"Códigos duplicados en múltiples categorías: {codigos_con_multiple_categoria}",
        details={"codigos_duplicados": codigos_con_multiple_categoria}
    )

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.3 Validar Agregación de RVL

# COMMAND ----------

print("🔍 Validando agregación de RVL...")
print("=" * 80)

# Agregar como en el código original
df_rvl_agregado = (
    df_rvl_con_agregador
    .groupBy("Detallista", "Material", "Fecha", "AgregadorCondicion", "claseOperacion")
    .agg(F.sum("MargenDetallado").alias("MargenTotal"))
    .withColumnRenamed("claseOperacion", "ClaseOperacionID")
)

# Test 1: Verificar que no se perdieron registros en la agregación
count_antes_agg = df_rvl_con_agregador.count()
count_despues_agg = df_rvl_agregado.count()

dq.log_test(
    "Agregación RVL - Registros Después de Group By",
    "PASS" if count_despues_agg > 0 else "FAIL",
    f"Antes: {count_antes_agg:,} | Después: {count_despues_agg:,}",
    details={
        "antes_agregacion": count_antes_agg,
        "despues_agregacion": count_despues_agg,
        "reduccion": count_antes_agg - count_despues_agg
    }
)

# Test 2: Verificar que MargenTotal no es null
dq.check_null_percentage(
    df_rvl_agregado,
    column="MargenTotal",
    threshold=0.0,
    test_name="Agregación RVL - MargenTotal No Null"
)

# Test 3: Verificar reconciliación de suma (suma antes == suma después)
suma_antes = df_rvl_con_agregador.agg(F.sum("MargenDetallado")).collect()[0][0] or 0
suma_despues = df_rvl_agregado.agg(F.sum("MargenTotal")).collect()[0][0] or 0
diferencia = abs(suma_antes - suma_despues)
tolerancia = 0.01  # Tolerancia de 1 centavo por redondeos

if diferencia <= tolerancia:
    dq.log_test(
        "Agregación RVL - Reconciliación de Suma",
        "PASS",
        f"Suma antes: {suma_antes:,.2f} | Suma después: {suma_despues:,.2f} | Diff: {diferencia:.2f}",
        details={
            "suma_antes": suma_antes,
            "suma_despues": suma_despues,
            "diferencia": diferencia,
            "tolerancia": tolerancia
        }
    )
else:
    dq.log_test(
        "Agregación RVL - Reconciliación de Suma",
        "FAIL",
        f"Diferencia en suma: {diferencia:,.2f} (tolerancia: {tolerancia})",
        details={
            "suma_antes": suma_antes,
            "suma_despues": suma_despues,
            "diferencia": diferencia,
            "tolerancia": tolerancia
        }
    )

# Test 4: Verificar que no hay duplicados en las claves
dq.check_duplicates(
    df_rvl_agregado,
    key_columns=["Detallista", "Material", "Fecha", "AgregadorCondicion", "ClaseOperacionID"],
    threshold=0.0,
    test_name="Agregación RVL - Sin Duplicados en Claves"
)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.4 Validar Ajuste de CostesDistribuidor

# COMMAND ----------

print("🔍 Validando ajuste de CostesDistribuidor...")
print("=" * 80)

# Calcular MargenTarifa con window function (como en código original)
windowSpec = Window.partitionBy("Detallista", "Material", "Fecha", "ClaseOperacionID")

df_rvl_con_margen_tarifa = (
    df_rvl_agregado
    .withColumn(
        "MargenTarifa",
        F.sum(
            F.when(F.col("AgregadorCondicion") == "Tarifa", F.col("MargenTotal"))
            .otherwise(0)
        ).over(windowSpec)
    )
)

# Test 1: Verificar que todas las particiones con CostesDistribuidor tienen MargenTarifa
costes_dist_sin_tarifa = (
    df_rvl_con_margen_tarifa
    .filter(F.col("AgregadorCondicion") == "CostesDistribuidor")
    .filter((F.col("MargenTarifa").isNull()) | (F.col("MargenTarifa") == 0))
    .count()
)

total_costes_dist = (
    df_rvl_con_margen_tarifa
    .filter(F.col("AgregadorCondicion") == "CostesDistribuidor")
    .count()
)

if costes_dist_sin_tarifa == 0:
    dq.log_test(
        "Ajuste CostesDistribuidor - Todos tienen MargenTarifa",
        "PASS",
        f"Todos los {total_costes_dist:,} registros CostesDistribuidor tienen MargenTarifa",
        details={"total_costes_distribuidor": total_costes_dist}
    )
else:
    porcentaje_sin_tarifa = (costes_dist_sin_tarifa / total_costes_dist * 100) if total_costes_dist > 0 else 0
    dq.log_test(
        "Ajuste CostesDistribuidor - Todos tienen MargenTarifa",
        "FAIL",
        f"{costes_dist_sin_tarifa:,} CostesDistribuidor sin MargenTarifa ({porcentaje_sin_tarifa:.2f}%)",
        severity="WARNING",
        details={
            "total_costes_distribuidor": total_costes_dist,
            "sin_margen_tarifa": costes_dist_sin_tarifa,
            "porcentaje": f"{porcentaje_sin_tarifa:.2f}%"
        }
    )

# Aplicar el ajuste
df_rvl_ajustado = (
    df_rvl_con_margen_tarifa
    .withColumn(
        "MargenTotal_Original",
        F.col("MargenTotal")  # Guardar original para comparación
    )
    .withColumn(
        "MargenTotal",
        F.when(
            F.col("AgregadorCondicion") == "CostesDistribuidor",
            F.col("MargenTotal") - F.col("MargenTarifa")
        ).otherwise(F.col("MargenTotal"))
    )
    .drop("MargenTarifa")
)

# Test 2: Verificar que el ajuste se aplicó correctamente
registros_ajustados = (
    df_rvl_ajustado
    .filter(F.col("AgregadorCondicion") == "CostesDistribuidor")
    .filter(F.col("MargenTotal") != F.col("MargenTotal_Original"))
    .count()
)

if registros_ajustados > 0:
    dq.log_test(
        "Ajuste CostesDistribuidor - Aplicado Correctamente",
        "PASS",
        f"{registros_ajustados:,} registros ajustados",
        details={
            "registros_ajustados": registros_ajustados,
            "total_costes_distribuidor": total_costes_dist
        }
    )
else:
    dq.log_test(
        "Ajuste CostesDistribuidor - Aplicado Correctamente",
        "FAIL",
        "No se aplicó ningún ajuste a CostesDistribuidor",
        severity="WARNING"
    )

# Test 3: Reconciliación de suma después del ajuste
suma_antes_ajuste = df_rvl_agregado.agg(F.sum("MargenTotal")).collect()[0][0] or 0
suma_despues_ajuste = df_rvl_ajustado.agg(F.sum("MargenTotal")).collect()[0][0] or 0

dq.log_test(
    "Ajuste CostesDistribuidor - Suma Cambió",
    "PASS",
    f"Suma antes: {suma_antes_ajuste:,.2f} | Suma después: {suma_despues_ajuste:,.2f}",
    details={
        "suma_antes": suma_antes_ajuste,
        "suma_despues": suma_despues_ajuste,
        "diferencia": suma_antes_ajuste - suma_despues_ajuste
    }
)

# Limpiar columna temporal
df_rvl_ajustado = df_rvl_ajustado.drop("MargenTotal_Original")

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.5 Validar Agregación de Ventas

# COMMAND ----------

print("🔍 Validando agregación de Ventas...")
print("=" * 80)

df_ventas_agregadas = (
    spark.table(Config.Tables.VENTAS)
    .join(F.broadcast(df_materiales_filtered), on="Material", how="inner")
    .groupBy("Detallista", "Material", "Fecha", "ClaseOperacionID")
    .agg(
        F.sum("VentaNeta").alias("VentaNeta"),
        F.sum("CantidadLitros").alias("CantidadLitros")
    )
)

# Test 1: Verificar que hay datos agregados
dq.check_record_count(
    df_ventas_agregadas,
    min_expected=Config.Thresholds.MIN_RECORDS_EXPECTED,
    test_name="Ventas Agregadas - Tiene Datos"
)

# Test 2: Verificar sin duplicados en claves
dq.check_duplicates(
    df_ventas_agregadas,
    key_columns=["Detallista", "Material", "Fecha", "ClaseOperacionID"],
    threshold=0.0,
    test_name="Ventas Agregadas - Sin Duplicados"
)

# Test 3: Reconciliación de suma de VentaNeta
suma_ventas_raw = (
    spark.table(Config.Tables.VENTAS)
    .join(F.broadcast(df_materiales_filtered), on="Material", how="inner")
    .agg(F.sum("VentaNeta"))
    .collect()[0][0] or 0
)

suma_ventas_agregadas = df_ventas_agregadas.agg(F.sum("VentaNeta")).collect()[0][0] or 0
diferencia = abs(suma_ventas_raw - suma_ventas_agregadas)

if diferencia <= 0.01:
    dq.log_test(
        "Ventas Agregadas - Reconciliación VentaNeta",
        "PASS",
        f"Raw: {suma_ventas_raw:,.2f} | Agregado: {suma_ventas_agregadas:,.2f}",
        details={
            "suma_raw": suma_ventas_raw,
            "suma_agregada": suma_ventas_agregadas,
            "diferencia": diferencia
        }
    )
else:
    dq.log_test(
        "Ventas Agregadas - Reconciliación VentaNeta",
        "FAIL",
        f"Diferencia: {diferencia:,.2f}",
        details={
            "suma_raw": suma_ventas_raw,
            "suma_agregada": suma_ventas_agregadas,
            "diferencia": diferencia
        }
    )

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.6 Validar Pivot de RVL

# COMMAND ----------

print("🔍 Validando pivot de RVL...")
print("=" * 80)

df_rvl_pivotado = (
    df_rvl_ajustado
    .groupBy("Detallista", "Material", "Fecha", "ClaseOperacionID")
    .pivot("AgregadorCondicion")
    .agg(F.sum("MargenTotal"))
    .fillna(0)
)

# Test 1: Verificar que no se perdieron registros en el pivot
count_antes_pivot = df_rvl_ajustado.select(
    "Detallista", "Material", "Fecha", "ClaseOperacionID"
).distinct().count()

count_despues_pivot = df_rvl_pivotado.count()

dq.compare_row_counts(
    df1=df_rvl_ajustado.select("Detallista", "Material", "Fecha", "ClaseOperacionID").distinct(),
    df2=df_rvl_pivotado,
    df1_name="Antes Pivot (Unique Keys)",
    df2_name="Después Pivot",
    threshold=0.0,
    test_name="Pivot RVL - Sin Pérdida de Registros"
)

# Test 2: Verificar que existen las columnas esperadas del pivot
columnas_esperadas = [
    "Tarifa", "Obsequios", "Promociones", "Descuentos", "Contratos",
    "AmortizacionesCDT", "Rappels", "CostesDistribuidor", "Costes"
]

columnas_pivotadas = [col for col in df_rvl_pivotado.columns if col not in [
    "Detallista", "Material", "Fecha", "ClaseOperacionID"
]]

columnas_faltantes = set(columnas_esperadas) - set(columnas_pivotadas)

if len(columnas_faltantes) == 0:
    dq.log_test(
        "Pivot RVL - Columnas Esperadas Creadas",
        "PASS",
        f"Todas las {len(columnas_esperadas)} columnas esperadas fueron creadas",
        details={"columnas_pivotadas": columnas_pivotadas}
    )
else:
    dq.log_test(
        "Pivot RVL - Columnas Esperadas Creadas",
        "FAIL",
        f"Faltan columnas: {list(columnas_faltantes)}",
        severity="WARNING",
        details={
            "columnas_esperadas": columnas_esperadas,
            "columnas_creadas": columnas_pivotadas,
            "columnas_faltantes": list(columnas_faltantes)
        }
    )

# Test 3: Verificar que fillna(0) se aplicó (no hay nulls)
null_counts = {}
for col in columnas_pivotadas:
    null_count = df_rvl_pivotado.filter(F.col(col).isNull()).count()
    null_counts[col] = null_count

total_nulls = sum(null_counts.values())

if total_nulls == 0:
    dq.log_test(
        "Pivot RVL - fillna(0) Aplicado",
        "PASS",
        "No hay nulls en las columnas pivotadas"
    )
else:
    dq.log_test(
        "Pivot RVL - fillna(0) Aplicado",
        "FAIL",
        f"Se encontraron {total_nulls} nulls en columnas pivotadas",
        details={"null_counts_por_columna": null_counts}
    )

# Test 4: Reconciliación de suma después del pivot
suma_antes_pivot = df_rvl_ajustado.agg(F.sum("MargenTotal")).collect()[0][0] or 0

# Sumar todas las columnas pivotadas
suma_expresion = sum([F.coalesce(F.col(col), F.lit(0)) for col in columnas_pivotadas])
suma_despues_pivot = df_rvl_pivotado.agg(F.sum(suma_expresion)).collect()[0][0] or 0

diferencia = abs(suma_antes_pivot - suma_despues_pivot)

if diferencia <= 0.01:
    dq.log_test(
        "Pivot RVL - Reconciliación de Suma",
        "PASS",
        f"Antes: {suma_antes_pivot:,.2f} | Después: {suma_despues_pivot:,.2f}",
        details={
            "suma_antes": suma_antes_pivot,
            "suma_despues": suma_despues_pivot,
            "diferencia": diferencia
        }
    )
else:
    dq.log_test(
        "Pivot RVL - Reconciliación de Suma",
        "FAIL",
        f"Diferencia en suma: {diferencia:,.2f}",
        details={
            "suma_antes": suma_antes_pivot,
            "suma_despues": suma_despues_pivot,
            "diferencia": diferencia
        }
    )

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.7 Validar Join Final: RVL Pivotado + Ventas

# COMMAND ----------

print("🔍 Validando join final: RVL Pivotado + Ventas...")
print("=" * 80)

df_final = df_rvl_pivotado.join(
    df_ventas_agregadas,
    on=["Detallista", "Material", "Fecha", "ClaseOperacionID"],
    how="left"
)

# Test 1: Verificar que no se perdieron registros de RVL en el join
dq.compare_row_counts(
    df1=df_rvl_pivotado,
    df2=df_final,
    df1_name="RVL Pivotado",
    df2_name="Final (después de join)",
    threshold=0.0,
    test_name="Join Final - Sin Pérdida de Registros RVL"
)

# Test 2: Verificar que hay registros con ventas
registros_con_ventas = df_final.filter(F.col("VentaNeta").isNotNull()).count()
total_registros = df_final.count()
porcentaje_con_ventas = (registros_con_ventas / total_registros * 100) if total_registros > 0 else 0

dq.log_test(
    "Join Final - Registros con Ventas",
    "PASS" if porcentaje_con_ventas > 50 else "FAIL",
    f"{registros_con_ventas:,} de {total_registros:,} tienen ventas ({porcentaje_con_ventas:.2f}%)",
    severity="WARNING" if porcentaje_con_ventas <= 50 else "INFO",
    details={
        "con_ventas": registros_con_ventas,
        "total": total_registros,
        "porcentaje": f"{porcentaje_con_ventas:.2f}%"
    }
)

# Test 3: Verificar que VentaNeta y CantidadLitros son consistentes
# Si hay VentaNeta, debería haber CantidadLitros
registros_venta_sin_litros = df_final.filter(
    (F.col("VentaNeta").isNotNull()) & (F.col("CantidadLitros").isNull())
).count()

if registros_venta_sin_litros == 0:
    dq.log_test(
        "Join Final - Consistencia VentaNeta-CantidadLitros",
        "PASS",
        "Todos los registros con VentaNeta tienen CantidadLitros"
    )
else:
    dq.log_test(
        "Join Final - Consistencia VentaNeta-CantidadLitros",
        "FAIL",
        f"{registros_venta_sin_litros:,} registros con VentaNeta pero sin CantidadLitros",
        severity="WARNING",
        details={"registros_inconsistentes": registros_venta_sin_litros}
    )

# COMMAND ----------

# MAGIC %md
# MAGIC ## 3️⃣ VALIDACIONES DE REGLAS DE NEGOCIO

# COMMAND ----------

# MAGIC %md
# MAGIC ### 3.1 Validar Rangos de Valores

# COMMAND ----------

print("🔍 Validando rangos de valores...")
print("=" * 80)

# Test 1: Verificar que no hay fechas fuera del rango esperado
fechas_fuera_rango = df_final.filter(
    (F.col("Fecha") < Config.FECHA_INICIO) | (F.col("Fecha") > Config.FECHA_FIN)
).count()

if fechas_fuera_rango == 0:
    dq.log_test(
        "Reglas Negocio - Fechas en Rango",
        "PASS",
        f"Todas las fechas están entre {Config.FECHA_INICIO} y {Config.FECHA_FIN}"
    )
else:
    dq.log_test(
        "Reglas Negocio - Fechas en Rango",
        "FAIL",
        f"{fechas_fuera_rango:,} registros con fechas fuera de rango",
        details={"registros_fuera_rango": fechas_fuera_rango}
    )

# Test 2: Verificar valores negativos en VentaNeta (puede ser válido, pero verificar)
ventas_negativas = df_final.filter(F.col("VentaNeta") < 0).count()
total_con_ventas = df_final.filter(F.col("VentaNeta").isNotNull()).count()
porcentaje_negativas = (ventas_negativas / total_con_ventas * 100) if total_con_ventas > 0 else 0

if porcentaje_negativas < 5:  # Menos del 5% es aceptable (devoluciones)
    dq.log_test(
        "Reglas Negocio - VentaNeta Negativas",
        "PASS",
        f"{ventas_negativas:,} ventas negativas ({porcentaje_negativas:.2f}%) - Aceptable para devoluciones",
        severity="INFO",
        details={
            "ventas_negativas": ventas_negativas,
            "total_ventas": total_con_ventas,
            "porcentaje": f"{porcentaje_negativas:.2f}%"
        }
    )
else:
    dq.log_test(
        "Reglas Negocio - VentaNeta Negativas",
        "FAIL",
        f"{ventas_negativas:,} ventas negativas ({porcentaje_negativas:.2f}%) - Revisar",
        severity="WARNING",
        details={
            "ventas_negativas": ventas_negativas,
            "total_ventas": total_con_ventas,
            "porcentaje": f"{porcentaje_negativas:.2f}%"
        }
    )

# Test 3: Verificar CantidadLitros siempre positiva cuando existe
litros_negativos = df_final.filter(
    F.col("CantidadLitros").isNotNull() & (F.col("CantidadLitros") < 0)
).count()

if litros_negativos == 0:
    dq.log_test(
        "Reglas Negocio - CantidadLitros Positiva",
        "PASS",
        "Todas las cantidades en litros son positivas"
    )
else:
    dq.log_test(
        "Reglas Negocio - CantidadLitros Positiva",
        "FAIL",
        f"{litros_negativos:,} registros con CantidadLitros negativa",
        details={"litros_negativos": litros_negativos}
    )

# COMMAND ----------

# MAGIC %md
# MAGIC ### 3.2 Validar Combinaciones de Claves

# COMMAND ----------

print("🔍 Validando combinaciones de claves...")
print("=" * 80)

# Test 1: Verificar que cada combinación Detallista+Material+Fecha es única por ClaseOperacionID
dq.check_duplicates(
    df_final,
    key_columns=["Detallista", "Material", "Fecha", "ClaseOperacionID"],
    threshold=0.0,
    test_name="Combinaciones Claves - Unicidad"
)

# Test 2: Verificar distribución de ClaseOperacionID
dist_clase_operacion = (
    df_final
    .groupBy("ClaseOperacionID")
    .agg(F.count("*").alias("count"))
    .orderBy(F.desc("count"))
)

print("\n📊 Distribución de ClaseOperacionID:")
display(dist_clase_operacion)

# Test 3: Verificar que hay múltiples fechas por Detallista+Material
combinaciones_con_multiples_fechas = (
    df_final
    .groupBy("Detallista", "Material")
    .agg(F.countDistinct("Fecha").alias("num_fechas"))
    .filter(F.col("num_fechas") > 1)
    .count()
)

total_combinaciones = df_final.select("Detallista", "Material").distinct().count()

if combinaciones_con_multiples_fechas > 0:
    dq.log_test(
        "Combinaciones Claves - Múltiples Fechas",
        "PASS",
        f"{combinaciones_con_multiples_fechas:,} combinaciones Detallista+Material tienen múltiples fechas",
        details={
            "con_multiples_fechas": combinaciones_con_multiples_fechas,
            "total_combinaciones": total_combinaciones
        }
    )
else:
    dq.log_test(
        "Combinaciones Claves - Múltiples Fechas",
        "FAIL",
        "No hay combinaciones con múltiples fechas - Posible problema",
        severity="WARNING"
    )

# COMMAND ----------

# MAGIC %md
# MAGIC ## 4️⃣ ANÁLISIS ESTADÍSTICO Y DETECCIÓN DE ANOMALÍAS

# COMMAND ----------

# MAGIC %md
# MAGIC ### 4.1 Estadísticas Descriptivas

# COMMAND ----------

print("📊 Calculando estadísticas descriptivas...")
print("=" * 80)

# Obtener columnas numéricas (columnas pivotadas + VentaNeta + CantidadLitros)
columnas_numericas = [col for col in df_final.columns if col not in [
    "Detallista", "Material", "Fecha", "ClaseOperacionID"
]]

# Calcular estadísticas
df_stats = df_final.select(
    *[
        F.mean(col).alias(f"{col}_mean")
        for col in columnas_numericas if col in df_final.columns
    ] +
    [
        F.stddev(col).alias(f"{col}_stddev")
        for col in columnas_numericas if col in df_final.columns
    ] +
    [
        F.min(col).alias(f"{col}_min")
        for col in columnas_numericas if col in df_final.columns
    ] +
    [
        F.max(col).alias(f"{col}_max")
        for col in columnas_numericas if col in df_final.columns
    ] +
    [
        F.expr(f"percentile({col}, 0.25)").alias(f"{col}_p25")
        for col in columnas_numericas if col in df_final.columns
    ] +
    [
        F.expr(f"percentile({col}, 0.50)").alias(f"{col}_median")
        for col in columnas_numericas if col in df_final.columns
    ] +
    [
        F.expr(f"percentile({col}, 0.75)").alias(f"{col}_p75")
        for col in columnas_numericas if col in df_final.columns
    ]
)
print("\n📊 Estadísticas por columna:")
display(df_stats)

# Crear resumen más legible
stats_dict = df_stats.first().asDict()
for col in columnas_numericas:
    if f"{col}_mean" in stats_dict:
        print(f"\n{col}:")
        print(f"  Mean: {stats_dict.get(f'{col}_mean', 0):,.2f}")
        print(f"  Std Dev: {stats_dict.get(f'{col}_stddev', 0):,.2f}")
        print(f"  Min: {stats_dict.get(f'{col}_min', 0):,.2f}")
        print(f"  25%: {stats_dict.get(f'{col}_p25', 0):,.2f}")
        print(f"  Median: {stats_dict.get(f'{col}_median', 0):,.2f}")
        print(f"  75%: {stats_dict.get(f'{col}_p75', 0):,.2f}")
        print(f"  Max: {stats_dict.get(f'{col}_max', 0):,.2f}")

# COMMAND ----------

# MAGIC %md
# MAGIC ### 4.2 Detección de Outliers (Z-Score)

# COMMAND ----------

print("🔍 Detectando outliers usando Z-Score...")
print("=" * 80)

# Para cada columna numérica, calcular z-score y detectar outliers
threshold = Config.Thresholds.OUTLIER_ZSCORE_THRESHOLD

for col in ["VentaNeta", "CantidadLitros"]:
    if col in df_final.columns:
        # Calcular mean y stddev
        stats = df_final.agg(
            F.mean(col).alias("mean"),
            F.stddev(col).alias("stddev")
        ).first()
        
        mean_val = stats["mean"]
        stddev_val = stats["stddev"]
        
        if stddev_val and stddev_val > 0:
            # Calcular z-score
            df_with_zscore = df_final.withColumn(
                f"{col}_zscore",
                F.abs((F.col(col) - mean_val) / stddev_val)
            )
            
            # Contar outliers
            outliers = df_with_zscore.filter(F.col(f"{col}_zscore") > threshold).count()
            total = df_with_zscore.filter(F.col(col).isNotNull()).count()
            outlier_percentage = (outliers / total * 100) if total > 0 else 0
            
            if outlier_percentage < 1:  # Menos del 1% es aceptable
                dq.log_test(
                    f"Outliers - {col}",
                    "PASS",
                    f"{outliers:,} outliers ({outlier_percentage:.2f}%) con |z-score| > {threshold}",
                    severity="INFO",
                    details={
                        "columna": col,
                        "outliers": outliers,
                        "total_no_null": total,
                        "porcentaje": f"{outlier_percentage:.2f}%",
                        "threshold_zscore": threshold
                    }
                )
            else:
                dq.log_test(
                    f"Outliers - {col}",
                    "FAIL",
                    f"{outliers:,} outliers ({outlier_percentage:.2f}%) - Revisar datos atípicos",
                    severity="WARNING",
                    details={
                        "columna": col,
                        "outliers": outliers,
                        "total_no_null": total,
                        "porcentaje": f"{outlier_percentage:.2f}%",
                        "threshold_zscore": threshold
                    }
                )
            
            # Mostrar algunos ejemplos de outliers
            if outliers > 0:
                print(f"\n📊 Ejemplos de outliers en {col}:")
                display(
                    df_with_zscore
                    .filter(F.col(f"{col}_zscore") > threshold)
                    .select("Detallista", "Material", "Fecha", col, f"{col}_zscore")
                    .orderBy(F.desc(f"{col}_zscore"))
                    .limit(10)
                )

# COMMAND ----------

# MAGIC %md
# MAGIC ### 4.3 Análisis de Tendencias Temporales

# COMMAND ----------

print("📈 Analizando tendencias temporales...")
print("=" * 80)

# Agregar por fecha para ver tendencias
df_tendencia = (
    df_final
    .groupBy("Fecha")
    .agg(
        F.count("*").alias("num_registros"),
        F.sum("VentaNeta").alias("venta_total"),
        F.sum("CantidadLitros").alias("litros_totales"),
        F.countDistinct("Detallista").alias("detallistas_unicos"),
        F.countDistinct("Material").alias("materiales_unicos")
    )
    .orderBy("Fecha")
)

print("\n📊 Tendencia por fecha:")
display(df_tendencia)

# Detectar días sin datos
todas_las_fechas = spark.sql(f"""
    SELECT date_add('{Config.FECHA_INICIO}', pos) as Fecha
    FROM (SELECT posexplode(split(repeat(',', datediff('{Config.FECHA_FIN}', '{Config.FECHA_INICIO}')), ',')))
""")

fechas_sin_datos = (
    todas_las_fechas
    .join(df_tendencia, on="Fecha", how="left_anti")
    .count()
)

if fechas_sin_datos == 0:
    dq.log_test(
        "Tendencias - Cobertura Diaria Completa",
        "PASS",
        "Hay datos para todas las fechas en el rango"
    )
else:
    total_dias = (datetime.strptime(Config.FECHA_FIN, "%Y-%m-%d") - 
                  datetime.strptime(Config.FECHA_INICIO, "%Y-%m-%d")).days + 1
    porcentaje_sin_datos = (fechas_sin_datos / total_dias * 100)
    
    dq.log_test(
        "Tendencias - Cobertura Diaria Completa",
        "FAIL",
        f"{fechas_sin_datos} días sin datos ({porcentaje_sin_datos:.2f}%)",
        severity="WARNING",
        details={
            "dias_sin_datos": fechas_sin_datos,
            "total_dias": total_dias,
            "porcentaje": f"{porcentaje_sin_datos:.2f}%"
        }
    )

# COMMAND ----------

# MAGIC %md
# MAGIC ## 5️⃣ RESUMEN FINAL Y REPORTE

# COMMAND ----------

# Mostrar resumen de todos los tests
dq.display_summary()

# COMMAND ----------

# MAGIC %md
# MAGIC ## 6️⃣ VALIDACIONES ADICIONALES RECOMENDADAS

# COMMAND ----------

# MAGIC %md
# MAGIC ### 6.1 Comparación con Ejecución Anterior (si existe)

# COMMAND ----------

print("📊 Validación de regresión (si existe tabla anterior)...")
print("=" * 80)

# Si tienes una tabla con la ejecución anterior, puedes comparar
# df_anterior = spark.table("damm_silver_des.analytics.rentabilidad_consolidada_anterior")
# 
# # Comparar conteos totales
# count_actual = df_final.count()
# count_anterior = df_anterior.count()
# diferencia_porcentaje = abs(count_actual - count_anterior) / count_anterior * 100
# 
# if diferencia_porcentaje < 10:  # Menos del 10% de cambio
#     print(f"✅ Conteo similar a ejecución anterior: {count_actual:,} vs {count_anterior:,}")
# else:
#     print(f"⚠️ Cambio significativo en conteo: {count_actual:,} vs {count_anterior:,} ({diferencia_porcentaje:.2f}%)")

print("ℹ️ Comparación con ejecución anterior no implementada (tabla anterior no disponible)")

# COMMAND ----------

# MAGIC %md
# MAGIC ### 6.2 Guardar Métricas de Calidad

# COMMAND ----------

print("💾 Guardando métricas de calidad...")
print("=" * 80)

# Crear DataFrame con métricas de calidad
metrics_data = []
timestamp = datetime.now()

# Agregar métricas clave
metrics_data.append({
    "metric_name": "total_records",
    "metric_value": float(df_final.count()),
    "timestamp": timestamp,
    "test_status": "PASS"
})

metrics_data.append({
    "metric_name": "total_tests_run",
    "metric_value": float(dq.test_count),
    "timestamp": timestamp,
    "test_status": "INFO"
})

metrics_data.append({
    "metric_name": "tests_passed",
    "metric_value": float(dq.passed_count),
    "timestamp": timestamp,
    "test_status": "PASS"
})

metrics_data.append({
    "metric_name": "tests_failed",
    "metric_value": float(dq.failed_count),
    "timestamp": timestamp,
    "test_status": "FAIL" if dq.failed_count > 0 else "PASS"
})

# Crear DataFrame
df_metrics = spark.createDataFrame(metrics_data)

print("📊 Métricas de calidad:")
display(df_metrics)
